In [ ]:
pip install twython

In [ ]:
pip install tweepy

In [ ]:
import os 
dir_name= os.getcwd()+'/data/'

In [ ]:
import tweepy

consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""


# authorization of consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)




In [ ]:
import pandas as pd
import os

#access the data folder
dir_name= os.getcwd()+'/data/'

# access the data folder to get the list of tickers in nasdaq
nasdaq_files=os.path.join(dir_name,'stock_ticker_datasets/nasdaq.csv')

# access the data folder to get the list of tickers in nyse
nyse_files=os.path.join(dir_name,'stock_ticker_datasets/nyse.csv')


nasdaq=pd.read_csv(nasdaq_files) 
nyse=pd.read_csv(nyse_files)

nasdaq['Symbol']=nasdaq['Symbol'].astype(str)
nasdaq_input=nasdaq['Symbol']


nyse['Symbol']=nyse['Symbol'].astype(str)
nyse_input=nyse['Symbol']

#chunk row size
n = 100

#split datasets into smaller sets to prevent blocked from the API
nasdaq_df = [nasdaq_input[i:i+n] for i in range(0,nasdaq_input.shape[0],n)]
nyse_df = [nyse_input[i:i+n] for i in range(0,nyse_input.shape[0],n)]


nasdaq.set_index("Symbol" , inplace=True)
nyse.set_index("Symbol" , inplace=True)



In [ ]:
import datetime, time
import csv

#  get data from the company's twitter timeline
# name: twitter screen name of the ticker
# number_of tweets: limit the number of tweets collected
# days: limit the number of days of the tweets

def returnTimelinetweets(name,number_of_tweets,days):

    alltweets = []
    user=api.get_user(name)
    userid=user.id_str
    # access the timeline tweets through the API
    timeline = api.user_timeline(user_id=userid,count=number_of_tweets)
    alltweets.extend(timeline)
    outtweets=[]
    for tweet in alltweets:
        if(datetime.datetime.now()-tweet.created_at).days<=days:
            outtweets=[tweet.created_at,tweet.text.encode("utf-8")]

    path=os.path.join(dir_name,'data-tweets/')
    with open(path+'data-'+name+'-tweets.csv','a') as f:
        writer= csv.writer(f)
        writer.writerows(outtweets)
            
# get data from  the cashtag
# name: twitter screen name of the ticker
# days: limit the number of days of the tweets
def get_tagtweets(name,days):
    print(name)
    # get the hasgtag/cashtag tweets
    hashtags=tweepy.Cursor(api.search,q=name,lang='en',tweet_mode='extended').items(200)
    # get the csv file of each ticker that contains the tweets
    path=os.path.join(dir_name,'data-tweets/'+'data-'+name+'-tweets.csv')
    print(path)
    try:
        with open(path,'a') as f:
            writer = csv.writer(f)
            for status in hashtags:
                if(datetime.datetime.now()-status.created_at).days<=days:
                    tweet_text = status.full_text.encode("utf-8")
                    # only collect the year, month and day
                    dates=str(status.created_at)[:10]
                    print(dates)
                    print(tweet_text)
                    writer.writerow([dates,tweet_text])
        
            sleep(2000)
    
    except Exception as e:
        print(e)
        pass


In [ ]:

import tweepy, datetime, time
import csv
import sys


# real time tweets collection using tweepy stream API
days=1
class MyStreamListener(tweepy.StreamListener):
    
    def __init__(self, output_file=sys.stdout,input_name=sys.stdout):
        super(MyStreamListener,self).__init__()
        self.output_file = output_file
        self.max_tweets=200
        self.tweet_count=0
        self.input_name=input_name
        self.days=days
        
    # override the on_status function
    def on_status(self, status):
        if(self.tweet_count==self.max_tweets):
            return False
        elif(datetime.datetime.now()-status.created_at).days<=self.days:
            tweet_text = status.text
            print(tweet_text)
            writer = csv.writer(self.output_file)
            
            writer.writerow([status.created_at,status.extended_tweet['full_text'].encode("utf-8")])
            self.tweet_count+=1
            
        
    def on_error(self, status_code):
        print(status_code)
        if status_code == 420:
            #returning False in on_error disconnects the stream
            
            return False
        
    def on_timeout(self):
        sys.stderr.write("Timeout, sleeping for 60 seconds...\n")
        time.sleep(60)
        return 
            


    
# loop through the list for each ticker

# for firm in list:

#         path=os.path.join(dir_name,'data-tweets/')
#         with open(path+'data-'+name+'-tweets.csv','a') as f:
#         writer = csv.writer(f)
#         myStreamListener = MyStreamListener(output_file=f,input_name=firm)
#         myStream = tweepy.Stream(auth = api.auth, tweet_mode='extended',listener=myStreamListener,languages = ["en"])
#         myStream.filter(track=firm)


In [ ]:
# update function to update the tweets using the get_tagtweets function
def update_tweets_all(day):
    for list in nasdaq_df:
        for name in list:
            name='$'+name
            get_tagtweets(name,day)

    for list in nyse_df:
        for name in list:
            name='$'+name
            get_tagtweets(name,day)
            
update_tweets_all(9)

In [ ]:
#return a pandas dataframe contains today's tweets
# name: name of the ticker

def get_today_tweets(name):
    text=[]
    tweets=pd.DataFrame()
    
    hashtags=tweepy.Cursor(api.search,q=name,lang='en',tweet_mode='extended').items(300)
    outttags=[] 
    path=os.path.join(dir_name,'data-tweets/')
    for status in hashtags:
        if(datetime.datetime.now()-status.created_at).days<=1:
            tweet_text = status.full_text.encode("utf-8")
            dates=str(status.created_at)[:10]
            text.append(tweet_text)
            dates.append(dates)
    tweets['date']=dates
    
    ticker=[name for i in range(len(dates))]
    tweets['ticker']=ticker
    tweets['text']=text
    
    return tweets
    
    
    
    

    



In [ ]:
# merging all the tweets csv file in  nasdaq
# name: name of the ticker
def merge_nasdaq_no_label_tweets(name):
    df = pd.DataFrame()
    for nas in nasdaq_df:
         for name in nas:
            try:
                # get the tweets file of each tciker
                path=os.path.join(dir_name,'data-tweets/'+'data-$'+name+'-tweets.csv')
                df_tweets = pd.read_csv(path ,names=['dates','tweets'],index_col='dates')
                # include the sector column in the pandas dataframe
                sector=nyse.loc[name]['Sector']
                ticker=[name for i in range(len(df_tweets))]
                Sector=[sector for i in range(len(df_tweets))]
                df_tweets['ticker']=ticker
                df_tweets['sector']=Sector
                df=df.append(df_tweets)


            except:
                print('no such file exist')
                pass

            df_path=os.path.join(dir_name,'train-data/nasdaq/nasdaq_no_labelled_tweets.csv')
            # output a csv file contains all the tweets in nasdaq
            df.to_csv(df_path)

In [ ]:
# merging all the tweets csv file in  nyse
# name: name of the ticker
def merge_nyse_no_label_tweets(name):
    df = pd.DataFrame()
    for nyse in nyse_df:
         for name in nyse:
            try:
                # get the tweets file of each tciker
                path=os.path.join(dir_name,'data-tweets/'+'data-$'+name+'-tweets.csv')
                df_tweets = pd.read_csv(path ,names=['dates','tweets'],index_col='dates')
                 # include the sector column in the pandas dataframe
                sector=nyse.loc[name]['Sector']
                ticker=[name for i in range(len(df_tweets))]
                Sector=[sector for i in range(len(df_tweets))]
                df_tweets['ticker']=ticker
                df_tweets['sector']=Sector
                df=df.append(df_tweets)


            except:
                print('no such file exist')
                pass

            df_path=os.path.join(dir_name,'train-data/nyse/nyse_no_labelled_tweets.csv')
            # output a csv file contains all the tweets in nyse
            df.to_csv(df_path)